# 4. Groebner Basis with Oscar

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("Oscar")

## Installation

**todo** instructions from email

`Oscar.jl` is a Julia wrapper for multiple computer algebra systems written in other languages:
- GAP
- Singular
- Polymake
- Nemo
- ...

**Note:** `Oscar.jl` runs on either MacOS or Linux. If you use Windows, you will need to install a Windows Subsystem for Linux ([WSL]). (this is a linux inside windows). Alternatively, the university has remote desktops you can log into in Linux [link]

In [1]:
using Oscar
Oscar.versioninfo()

OSCAR version 1.5.0
  combining:
    AbstractAlgebra.jl   v0.47.3
    GAP.jl               v0.15.3
    Hecke.jl             v0.38.6
    Nemo.jl              v0.52.1
    Polymake.jl          v0.13.2
    Singular.jl          v0.26.0


### Overview

Oscar covers lots of different areas of computational algebra, including:
- Group theory
- Number theory
- Commutative algebra
- Polyhedral geometry
- Toric geometry
- Algebraic geometry
- and more...

There is much more that can be done than will be covered here.
See the list of online tutorials for Oscar [here](https://www.oscar-system.org/tutorials/).

## Polynomial rings

In [25]:
ZZ(5)

5

In [69]:
R, x = polynomial_ring(QQ, "x")

(Univariate polynomial ring in x over QQ, x)

In [70]:
f = x^2 + x + 1
f(2)

7

In [ ]:
R, (x, y) = QQ[:x, :y]  # shorthand

f = x^2 + y^3
g = y^2 - x + 4

@show f(2, 3)
@show g(1, f)

f(2, 3) = 31
g(1, f) = x^4 + 2*x^2*y^3 + y^6 + 3


x^4 + 2*x^2*y^3 + y^6 + 3

In [55]:
Z2, x = finite_field(2, "x")
R, y = Z2[:y]

f = y + 1
f(1)

0

In [82]:
S, x, y = polynomial_ring(ZZ, :x => (1:2, 1:2), :y => 1:3)
@show x
@show y

x = ZZMPolyRingElem[x[1, 1] x[1, 2]; x[2, 1] x[2, 2]]
y = ZZMPolyRingElem[y[1], y[2], y[3]]


3-element Vector{ZZMPolyRingElem}:
 y[1]
 y[2]
 y[3]

There are many other ring constructors available (see this [list](https://nemocas.github.io/Nemo.jl/stable/constructors/))

In [ ]:
@show ZZ
@show QQ
@show QQBar
@show real_field()
@show complex_field()
@show fraction_field(ZZ)
@show finite_field(5)
@show matrix_space(QQ, 2, 2)

ZZ = Integer ring
QQ = Rational field
QQBar = Algebraic closure of rational field
real_field() = Real field
complex_field() = Complex field
fraction_field(ZZ) = Rational field
finite_field(5) = (Prime field of characteristic 5, 0)
matrix_space(QQ, 2, 2) = Matrix space of 2 rows and 2 columns over QQ


Matrix space of 2 rows and 2 columns
  over rational field

## Ideals
Let $K[x]$ be a polynmoial ring over field $K$. We can define a **polynomial Ideal** $I = \{f_1,...,f_s\}$ where $f \in K[x]$. It has been shown that one can define a direct relationship between ideals and varieties.
For our purposes then, ideals represent the *solution set* of a set of polynomials $[f_1,...,f_s]^\top = 0$.

See the [documentation](https://docs.oscar-system.org/stable/CommutativeAlgebra/ideals/) for a full list of features for ideals in Oscar.

In [152]:
R, (x,y) = polynomial_ring(QQ, ["x","y"])

polys = [x^2+-y, x*y^2 -y]

I = ideal(R, polys)

Ideal generated by
  x^2 - y
  x*y^2 - y

In [153]:
@show base_ring(I)
@show gens(I)
@show dim(I)

base_ring(I) = Multivariate polynomial ring in 2 variables over QQ
gens(I) = QQMPolyRingElem[x^2 - y, x*y^2 - y]
dim(I) = 0


0

In [154]:
J = ideal(R, [x, y])^3

Ideal generated by
  x^3
  x^2*y
  x*y^2
  y^3

In [ ]:
@show I + J
@show I * J
@show intersect(I, J)
@show quotient(I, J)

I + J = Ideal (x^2 - y, x*y^2 - y, x^3, x^2*y, x*y^2, y^3)
I * J = Ideal with 8 generators
intersect(I, J) = Ideal (x^3 - y^3, -x*y^2 + y^4, -x^2*y + x*y^3, -x^3 + x^2*y^2)
quotient(I, J) = Ideal (-x + y^2, x*y - 1, x^2 - y)


Ideal generated by
  -x + y^2
  x*y - 1
  x^2 - y

Several algorithms for ideals are implemented. Note that the following methods compute a Groebner basis in the background.

In [ ]:
eliminate(I, [x])  # eliminate x

Ideal generated by
  y^4 - y

In [157]:
f = x^3
ideal_membership(f, J)

true

In [ ]:
rational_solutions(I)  # see also real_solutions()

2-element Vector{Vector{QQFieldElem}}:
 [1, 1]
 [0, 0]

## Groebner Basis

A **Groebner Basis** $G = \{g_1,...,g_r\}$ is a useful representation of a polynomial ideal $I$ (for some *monomial ordering*). For example, we can try to find $G$ which projects the set of polynomials in $I$ to a system with fewer variables (i.e. variable elimination, similar to Gaussian elimination in linear algebra).

In [161]:
R, (x,y) = polynomial_ring(QQ, ["x","y"])
I = ideal(R, [x^2+-y, x*y^2 -y])
groebner_basis(I)

Gröbner basis with elements
  1: x^2 - y
  2: y^3 - x*y
  3: x*y^2 - y
with respect to the ordering
  degrevlex([x, y])

The choice of $g \in G$ is dependent on the *monomial ordering* of the polynomials. Common choices are:
- Lexicographical (`lex`) for elimination
- Degree Reverse Lexicographical (`degrevlex`) for efficient computation of Groebner basis (default)

In [163]:
# can pick a different ordering (degrevlex by default)
groebner_basis(I, ordering=lex(R))

Gröbner basis with elements
  1: y^4 - y
  2: x*y - y^3
  3: x^2 - y
with respect to the ordering
  lex([x, y])

In [164]:
# if dim=0, can use FGLM algorithm
@assert dim(I) == 0

fglm(I, start_ordering=degrevlex(R), destination_ordering=lex(R))

groebner_basis(I, algorithm=:fglm, ordering=lex(R))  # equivalent

Gröbner basis with elements
  1: y^4 - y
  2: x*y - y^3
  3: x^2 - y
with respect to the ordering
  lex([x, y])

In [165]:
sol = rational_solutions(I)

2-element Vector{Vector{QQFieldElem}}:
 [0, 0]
 [1, 1]

In [166]:
# if not finite dim
groebner_walk(I, lex(R), degrevlex(R))

Gröbner basis with elements
  1: y^4 - y
  2: x*y - y^3
  3: x^2 - y
with respect to the ordering
  lex([x, y])

### Fraction field parameters
Finding a GB where the parameters (coefficients) are not known

In [167]:
# Ring of coefficients
C, a = polynomial_ring(QQ, "a" => 1:6)

# Fraction field of coefficients
F = fraction_field(C)

# Ring of polynomials with coefficients in F
R, (x, y) = polynomial_ring(F, ["x", "y"])

(Multivariate polynomial ring in 2 variables over F, AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}[x, y])

In [168]:
polys = [
    a[1]*x^2 - a[2]*y - a[3],
    a[4]*y^2 + a[5]*(x - y) + a[6]
]
I = ideal(R, polys)

Ideal generated by
  a[1]*x^2 - a[2]*y - a[3]
  a[5]*x + a[4]*y^2 - a[5]*y + a[6]

In [169]:
groebner_basis(I, algorithm=:fglm, ordering=lex(R))

Gröbner basis with elements
  1: -a[1]*a[4]^2*y^4 + 2*a[1]*a[4]*a[5]*y^3 + (-2*a[1]*a[4]*a[6] - a[1]*a[5]^2)*y^2 + (2*a[1]*a[5]*a[6] + a[2]*a[5]^2)*y - a[1]*a[6]^2 + a[3]*a[5]^2
  2: -a[5]*x - a[4]*y^2 + a[5]*y - a[6]
with respect to the ordering
  lex([x, y])